In [ ]:
!pip install polars

In [ ]:
import polars as pl
from datetime import datetime

In [ ]:
filtered_user_video_df = pl.read_parquet("/kaggle/input/video-user-table/user_video_new.parquet")
filtered_user_video_df.head()

In [ ]:
course_info_limit_df = pl.read_csv("/kaggle/input/merge-problem/course_info_limit.csv")
course_info_limit_df.head()

In [ ]:
user_df = pl.read_ndjson("/kaggle/input/lightmooccubex/entities/user.json")
user_df.head()

In [ ]:
import requests

url = "https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/video.json"
video_df = pl.read_ndjson(url)
video_df.head()

In [ ]:
course_df = pl.read_ndjson("/kaggle/input/lightmooccubex/entities/course.json")
course_df.head()

In [ ]:
#GIỮ LẠI CÁC COURSE GIỚI HẠN MÀ USER CÓ THỂ ĐKI, BỎ CÁC ENROLL_TIME TƯƠNG ỨNG NẾU BỎ KHÓA HỌC ĐÓ
#Chuẩn hóa course_info_limit_df.course_id về dạng i64 (bỏ "C_")
valid_course_ids = course_info_limit_df.select(
    pl.col("course_id").str.strip_prefix("C_").cast(pl.Int64)
).to_series()


# Cần: valid_course_ids là một Series, chuyển về set để tra nhanh
valid_course_ids_set = set(valid_course_ids)

# Bước 1: Tạo mask hợp lệ (True/False) cho từng phần tử trong list
user_df = user_df.with_columns([
    pl.col("course_order").list.eval(
        pl.element().is_in(valid_course_ids_set)
    ).alias("valid_mask")
])

# Bước 2: Lọc cả course_order và enroll_time theo mask
user_df = user_df.with_columns([
    pl.struct(["course_order", "valid_mask"]).map_elements(
        lambda x: [cid for cid, keep in zip(x["course_order"], x["valid_mask"]) if keep]
    ).alias("filtered_course_order"),

    pl.struct(["enroll_time", "valid_mask"]).map_elements(
        lambda x: [et for et, keep in zip(x["enroll_time"], x["valid_mask"]) if keep]
    ).alias("filtered_enroll_time")
])

# Bước 3: Dọn dẹp mask và ghi đè enroll_time nếu muốn
user_df = user_df.drop("valid_mask").with_columns(
    pl.col("filtered_enroll_time").alias("enroll_time")
)

# Bước 4: Lọc bỏ dòng có filtered_course_order là list rỗng
user_df = user_df.filter(pl.col("filtered_course_order").list.len() > 0)

# Kiểm tra kết quả
user_df.head()


In [ ]:
# DICT ÁNH XẠ VIDEO ID VÀ CCID
video_to_ccid = {}

with open("/kaggle/input/lightmooccubex/relations/video_id-ccid.txt", "r") as file:
    for line in file:
        video_id, ccid = line.strip().split("\t")
        video_to_ccid[video_id] = ccid

In [ ]:
# Bỏ cột resourse dạng string cũ
course_info_limit_df = course_info_limit_df.drop('resource')

In [ ]:
#chuyển resource về lại dạng struct để lọc ra video_id
course_info_limit_df = course_info_limit_df.join(
    course_df.select([
        pl.col("id").alias("course_id"),  # đổi tên để khớp
        pl.col("resource")
    ]),
    on="course_id",
    how="left"
)

In [ ]:
#trong course_info_limit_df lọc ra video id của khóa học đó vào cột video_ids
course_info_limit_df = course_info_limit_df.with_columns([
    pl.col("resource")
    .list.eval(
        pl.when(pl.element().struct.field("resource_id").str.starts_with("V_"))
        .then(pl.element().struct.field("resource_id"))
        .otherwise(None)
    )
    .list.drop_nulls()
    .alias("video_ids")
])

course_info_limit_df[:2,: ]

In [ ]:
# TẠO CỘT CCID ỨNG VỚI VIDEO ID CỦA COURSE ĐÓ
course_info_limit_df = course_info_limit_df.with_columns(
    pl.col("video_ids").map_elements(
        lambda vids: [video_to_ccid.get(v, None) for v in vids],
        return_dtype=pl.List(pl.Utf8)
    ).alias("ccids")
)

course_info_limit_df[:2,: ]

In [ ]:
# TẠO CỘT CCID ỨNG VỚI VIDEO ID MÀ MỖI USER ĐÃ XEM
filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.col("user_videos_id").map_elements(
        lambda vids: [video_to_ccid.get(v, None) for v in vids],
        return_dtype=pl.List(pl.Utf8)
    ).alias("user_video_ccids")
)

filtered_user_video_df[:2,: ]

In [ ]:
course_info_limit_df = course_info_limit_df.with_columns([
    pl.col("course_id").str.replace("C_", "").alias("clean_course_id")
])

#tạo ra 1 dict sao cho với key là video ccid, value là course id đã bỏ C_ 
ccid_to_course = {}

for row in course_info_limit_df.iter_rows(named=True):
    course_id = row["course_id"].removeprefix("C_")
    ccids = row["ccids"]
    for ccid in ccids:
        ccid_to_course[ccid] = course_id

In [ ]:
ccid_to_course["50A52A74EE3C59CB9C33DC5901307461"]

In [ ]:
course_info_limit_df[:2,: ]

In [ ]:
# TẠO CỘT COURSE_OF_WATCHED_VID ỨNG VỚI VIDEO CCID MÀ MỖI USER ĐÃ XEM
filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.col("user_video_ccids").map_elements(
        lambda ccids: [ccid_to_course.get(ccid, None) for ccid in ccids],
        return_dtype=pl.List(pl.Utf8)
    ).alias("course_of_watched_video")
)

filtered_user_video_df[:2,: ]

In [ ]:
# KIỂM TRA USER XEM VIDEO CÓ CCID KO NẰM TRONG COURSE ĐÃ GIỚI HẠN
all_null_count = filtered_user_video_df.filter(
    pl.col("course_of_watched_video").list.eval(pl.element().is_null()).list.all()
).height

mixed_count = filtered_user_video_df.filter(
    pl.col("course_of_watched_video").list.eval(pl.element().is_null()).list.any() &
    ~pl.col("course_of_watched_video").list.eval(pl.element().is_null()).list.all()
).height

all_valid_count = filtered_user_video_df.filter(
    pl.col("course_of_watched_video").list.eval(pl.element().is_not_null()).list.all()
).height

print("Toàn null:", all_null_count)
print("Vừa null vừa số:", mixed_count)
print("Toàn số:", all_valid_count)


In [ ]:
# BỎ CÁC HÀNG CÓ COURSE ID ỨNG VỚI VIDEO ĐÃ XEM TOÀN NULL
filtered_user_video_df = filtered_user_video_df.filter(
    ~pl.col("course_of_watched_video").list.eval(pl.element().is_null()).list.all()
)

# BỎ CÁC PHẦN TỬ CÓ COURSE ID ỨNG VỚI VIDEO ĐÃ XEM CỦA 1 USER LÀ NULL
# đối với các hàng có null, ví dụ [null,  "1761386", … "1761386"] thì null nằm ở đầu,
# bỏ null này nhưng đồng nghĩa phải bỏ hết các phần tử đầu của list ở các cột khác, 
# vì các cột có dạng đều là list và mỗi phần tử song song với nhau về vị trí

def drop_elements_by_mask(df: pl.DataFrame, mask_col: str, exclude_cols: list[str] = ["user_id"]) -> pl.DataFrame:
    """
    Loại bỏ các phần tử tại các vị trí có null trong cột mask_col,
    áp dụng cho tất cả các cột dạng list (trừ exclude_cols và cột mask).

    Args:
        df: pl.DataFrame
        mask_col: tên cột list dùng để mask
        exclude_cols: danh sách các cột không xử lý (mặc định chứa "user_id")

    Returns:
        pl.DataFrame đã xử lý
    """
    list_cols = [
        col for col, dtype in df.schema.items()
        if isinstance(dtype, pl.List) and col not in exclude_cols and col != mask_col
    ]

    for col in list_cols:
        inner_dtype = df.schema[col].inner
        df = df.with_columns(
            pl.struct([col, mask_col]).map_elements(
                lambda row: [
                    val for val, keep in zip(row[col], row[mask_col]) if keep is not None
                ],
                return_dtype=pl.List(inner_dtype)
            ).alias(col)
        )

    # Xử lý riêng cột mask
    if mask_col not in exclude_cols:
        df = df.with_columns(
            pl.col(mask_col).list.drop_nulls().alias(mask_col)
        )

    return df

filtered_user_video_df = drop_elements_by_mask(filtered_user_video_df, "course_of_watched_video", exclude_cols=["user_id"])


In [ ]:
# KIỂM TRA LẠI USER XEM VIDEO CÓ CCID KO NẰM TRONG COURSE ĐÃ GIỚI HẠN
all_null_count = filtered_user_video_df.filter(
    pl.col("course_of_watched_video").list.eval(pl.element().is_null()).list.all()
).height

mixed_count = filtered_user_video_df.filter(
    pl.col("course_of_watched_video").list.eval(pl.element().is_null()).list.any() &
    ~pl.col("course_of_watched_video").list.eval(pl.element().is_null()).list.all()
).height

all_valid_count = filtered_user_video_df.filter(
    pl.col("course_of_watched_video").list.eval(pl.element().is_not_null()).list.all()
).height

print("Toàn null:", all_null_count)
print("Vừa null vừa số:", mixed_count)
print("Toàn số:", all_valid_count)


In [ ]:
# Từ bảng user_df, tạo một dict với user_id làm key, 
# value là dict ánh xạ từ course_id → enroll_time.
# Áp dụng hàm ánh xạ vào bảng filtered_user_video_df:
# Với mỗi user_id và danh sách course_of_watched_video, 
# ánh xạ từng course_id sang thời gian đăng ký (nếu có).

# 1. Tạo dict: {user_id: {course_id: enroll_time}}
user_to_course_time = {}

for row in user_df.iter_rows(named=True):
    user_id = row["id"]
    # Ép kiểu course_id về str để match với course_of_watched_video
    course_ids = [str(cid) for cid in row["filtered_course_order"]]
    enroll_times = row["filtered_enroll_time"]
    course_time_map = dict(zip(course_ids, enroll_times))
    user_to_course_time[user_id] = course_time_map


# 2. Thêm cột enroll_time tương ứng với course_of_watched_video
filtered_user_video_df = filtered_user_video_df.with_columns(
    pl.struct(["user_id", "course_of_watched_video"]).map_elements(
        lambda row: [
            user_to_course_time.get(row["user_id"], {}).get(cid, None)
            for cid in row["course_of_watched_video"]
        ],
        return_dtype=pl.List(pl.Utf8)  # hoặc pl.Datetime nếu là thời gian thực
    ).alias("enroll_time")
)

filtered_user_video_df[:2,: ]

In [ ]:
# KIỂM TRA COURSE ID CỦA VIDEO USER XEM CÓ ENROLL TIME HAY KHÔNG
all_null_count = filtered_user_video_df.filter(
    pl.col("enroll_time").list.eval(pl.element().is_null()).list.all()
).height

mixed_count = filtered_user_video_df.filter(
    pl.col("enroll_time").list.eval(pl.element().is_null()).list.any() &
    ~pl.col("enroll_time").list.eval(pl.element().is_null()).list.all()
).height

all_valid_count = filtered_user_video_df.filter(
    pl.col("enroll_time").list.eval(pl.element().is_not_null()).list.all()
).height

print("Toàn null:", all_null_count)
print("Vừa null vừa số:", mixed_count)
print("Toàn số:", all_valid_count)


In [ ]:
filtered_user_video_df = filtered_user_video_df.filter(
    ~pl.col("enroll_time").list.eval(pl.element().is_null()).list.all()
)

# XÓA CÁC HÀNG TOÀN NULL, PHẦN TỬ NULL
filtered_user_video_df = drop_elements_by_mask(filtered_user_video_df, "enroll_time", exclude_cols=["user_id"])

In [ ]:
# KIỂM TRA LẠI COURSE ID CỦA VIDEO USER XEM CÓ ENROLL TIME HAY KHÔNG
all_null_count = filtered_user_video_df.filter(
    pl.col("enroll_time").list.eval(pl.element().is_null()).list.all()
).height

mixed_count = filtered_user_video_df.filter(
    pl.col("enroll_time").list.eval(pl.element().is_null()).list.any() &
    ~pl.col("enroll_time").list.eval(pl.element().is_null()).list.all()
).height

all_valid_count = filtered_user_video_df.filter(
    pl.col("enroll_time").list.eval(pl.element().is_not_null()).list.all()
).height

print("Toàn null:", all_null_count)
print("Vừa null vừa số:", mixed_count)
print("Toàn số:", all_valid_count)


In [ ]:
filtered_user_video_df.head()

In [ ]:
# Hàm đếm số hàng có độ dài phần tử bằng nhau giữa các cột (trừ user_id):

def count_rows_with_equal_list_lengths(df: pl.DataFrame, exclude_cols=["user_id"]) -> int:
    list_cols = [col for col in df.columns if col not in exclude_cols]
    count = 0

    for row in df.iter_rows(named=True):
        lengths = [len(row[col]) if isinstance(row[col], list) else -1 for col in list_cols]
        if len(set(lengths)) == 1:
            count += 1

    return count

matched_rows = count_rows_with_equal_list_lengths(filtered_user_video_df)
print(f"Số hàng có độ dài các list bằng nhau: {matched_rows}")


In [ ]:
print(filtered_user_video_df.shape)
print(user_df.shape)

In [ ]:
filtered_user_video_df.write_parquet("uv_filtered_course.parquet") 
user_df.write_parquet("user_filtered_course.parquet") 
course_info_limit_df.write_parquet("course_info_limit.parquet") 

In [ ]:
user_df.head()

In [ ]:
course_info_limit_df.head()